### 국내 강/하천 목자판 수위 레벨 인식

<br>

[수위 측정 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=71494)
<br>[국내 강/하천 목자판 수위 레벨 인식](https://aifactory.space/task/2639/leaderboard)

<br>

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [7]:
def create_model():

    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    """
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(128, 128, 3)),
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer

        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer
        tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
        tf.keras.layers.BatchNormalization(), #batch norm layer

        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(1) # ouput layer
        ])

    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])
    return model

def main(x_npy_path, y_npy_path, model_save_path):
    # Load data
    X = np.load(x_npy_path)
    y = np.load(y_npy_path)

    # Split data into training and validation set
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    model = create_model()
    model.fit(X_train, y_train, epochs=3, batch_size=8, validation_data=(X_val, y_val), verbose=1)
    loss = model.evaluate(X_val, y_val)
    # print(f"Test loss: {loss}")
    # Save the model
    model.save(model_save_path)

if __name__ == '__main__':
    """
    import sys

    if len(sys.argv) != 4:
        print("Usage: python train.py [path_to_x.npy] [path_to_y.npy] [path_to_model.h5]")
        sys.exit(1)
    """
    x_npy_path = 'data/datasets/x_train.npy'
    y_npy_path = 'data/datasets/y_train.npy'
    model_save_path = 'output'
    main(x_npy_path, y_npy_path, model_save_path)

Epoch 1/3


2023-11-13 00:10:46.035179: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_4/dropout_19/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2284/2284 [==============================] - 63s 26ms/step - loss: 20.8080 - mae: 3.7859 - val_loss: 15.3633 - val_mae: 3.3515
Epoch 2/3
2284/2284 [==============================] - 59s 26ms/step - loss: 15.8237 - mae: 3.4009 - val_loss: 17.1760 - val_mae: 3.4929
Epoch 3/3
143/143 [==============================] - 3s 20ms/step - loss: 14.7341 - mae: 3.2905
INFO:tensorflow:Assets written to: output/assets


INFO:tensorflow:Assets written to: output/assets


In [8]:
import numpy as np
import pandas as pd
import sys
from keras.models import load_model

In [9]:
def predict_and_save(model_path, x_test_path, y_pred_path):
    # 모델 불러오기
    model = load_model(model_path)

    # x_test.npy 파일에서 데이터 로드
    x_test = np.load(x_test_path)

    # 예측 수행
    y_pred = model.predict(x_test)

    # 예측 결과를 y_pred.npy로 저장
    np.save(y_pred_path, y_pred)

if __name__ == "__main__":

    """
    if len(sys.argv) != 4:
        print("Usage: python3 model_inference.py <model_path> <x_test_path> <y_pred_path>")
        sys.exit(1)
    """
    
    model_path = 'output'   # 모델 파일 경로
    x_test_path = 'data/datasets/x_test.npy'   # 테스트 데이터 파일 경로
    y_pred_path = 'y_pred.npy'   # 예측 결과 저장 파일 경로

    predict_and_save(model_path, x_test_path, y_pred_path)

179/179 [==============================] - 3s 16ms/step


In [10]:
y_pred_npy_path = 'y_pred.npy'
y_pred = np.load(y_pred_npy_path)

# Convert the numpy array to a Pandas DataFrame
y_pred_df = pd.DataFrame(y_pred)

# Save the DataFrame to a CSV file
y_pred_csv_path = y_pred_npy_path.replace('.npy', '.csv')
y_pred_df.to_csv(y_pred_csv_path, index=False)

# 저장된 .csv 파일의 경로를 출력합니다.
print(f'Saved CSV to {y_pred_csv_path}')

Saved CSV to y_pred.csv
